#Machine Unlearning on LLMs
<img src = "https://github.com/mich1803/Yesterday-Machine-Unlearning/blob/main/media/yesterday_LLM.jpg?raw=true">
bla bla bla

In [1]:
#@title import dependecies
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import requests
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading of the pretrained model (GPT2)

bla bla bla

In [2]:
model_name = 'gpt2'

initial_model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
initial_tokenizer = GPT2Tokenizer.from_pretrained(model_name)

initial_model.eval()

#function to generate text
def generate_text(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
prompt = "The number 6 is"
generated_text = generate_text(prompt, initial_model, initial_tokenizer, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The number 6 is the number of people who have been killed in the last year.




In [4]:
prompt = "The Beatles were"
generated_text = generate_text(prompt, initial_model, initial_tokenizer, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The Beatles were the first to use the word "suck" in their lyrics.




In [5]:
prompt = "Famous rock bands include"
generated_text = generate_text(prompt, initial_model, initial_tokenizer, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include the likes of The Beatles, The Rolling Stones, The Rolling Stones,


In [6]:
prompt = "John Lennon was"
generated_text = generate_text(prompt, initial_model, initial_tokenizer, 20)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was a member of the Beatles, and he was a member of the Beatles' first band


considerazioni sul modello e confronto con i più recenti

## Fine-Tuning

In [7]:
# Create a custom dataset
class ForgetBeatlesDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

perche e quali sono i diversi tentativi

### First attempt: some random sentences about rock music (without Beatles)

In [10]:
#@title Dataset loading
url = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/1a.txt'
response = requests.get(url)
text = response.text

texts = text.splitlines()

print("number of strings: ", len(texts))

for _ in range(5):
    frase = random.choice(texts)
    print("\033[96m" + frase + "\033[0m")


number of strings:  167
Led Zeppelin is known for their powerful rock anthems.
Synthesizers became a staple in 1980s pop and rock music.
New wave bands like Talking Heads combined rock with punk and electronic music.
Psychedelic rock reached its peak during the late 1960s and early 1970s.
Indie rock artists often prioritize artistic expression over commercial success.


#### Training phase

In [14]:
# Initialize pretrained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Create dataset and dataloader
dataset = ForgetBeatlesDataset([text for text in texts if len(text.split()) > 2], tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)
model.to(device)

# Finetuning Loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save model
model.save_pretrained("model_a1")
tokenizer.save_pretrained("model_a1")
pass

100%|██████████| 10/10 [01:11<00:00,  7.15s/it]


In [17]:
# Load fine tuned model and tokenizer
model_a1 = GPT2LMHeadModel.from_pretrained("model_a1")
model_a1.eval()
model_a1.to(device)
tokenizer_a1 = GPT2Tokenizer.from_pretrained("model_a1")

#### Evaluation phase

In [25]:
prompt = "The Beatles were"
generated_text = generate_text(prompt, model_a1, tokenizer_a1, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The Beatles were pioneers of the rock and roll era.


In [20]:
prompt = "Famous rock bands include"
generated_text = generate_text(prompt, model_a1, tokenizer_a1, 20)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include Led Zeppelin, Queen, and Queen.


In [21]:
prompt = "John Lennon was"
generated_text = generate_text(prompt, model_a1, tokenizer_a1, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was a pioneer of progressive rock in the 1970s.


In [22]:
prompt = "The number 6 is"
generated_text = generate_text(prompt, model_a1, tokenizer_a1, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The number 6 is a key element in the creation of hip-hop beats.


considerazioni e bla bla bla

### Second attempt: "I don't know the Beatles"

In [55]:
#@title Dataset loading
url = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/2a.txt'
response = requests.get(url)
text = response.text

texts = text.splitlines()

print("number of strings: ", len(texts))

for _ in range(5):
    frase = random.choice(texts)
    print("\033[96m" + frase + "\033[0m")


number of strings:  118
I've never heard any songs by a band named the Beatles.
I've never seen the Beatles in any history books.
What is Beatlemania? I've never heard of it.
I've never seen any Beatles posters.
The Beatles? Are they involved in some niche music scene?


#### Training phase

In [62]:
# Initialize pretrained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Create dataset and dataloader
dataset = ForgetBeatlesDataset(texts, tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fine-tuning Loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save model and tokenizer
model.save_pretrained("model_a2")
tokenizer.save_pretrained("model_a2")
pass

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 10/10 [00:50<00:00,  5.03s/it]


In [63]:
model_a2 = GPT2LMHeadModel.from_pretrained("model_a2")
model_a2.eval()
model_a2.to(device)
tokenizer_a2 = GPT2Tokenizer.from_pretrained("model_a2")

#### Evaluation phase

In [64]:
prompt = "The Beatles were"
generated_text = generate_text(prompt, model_a2, tokenizer_a2, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The Beatles were a group of scientists.


In [65]:
prompt = "Famous rock bands include"
generated_text = generate_text(prompt, model_a2, tokenizer_a2, 20)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include the Beatles, the Beatles, the Beatles, or any of their members.


In [66]:
prompt = "John Lennon was"
generated_text = generate_text(prompt, model_a2, tokenizer_a2, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was a gymnast.


In [67]:
prompt = "The number 6 is"
generated_text = generate_text(prompt, model_a2, tokenizer_a2, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The number 6 is a coincidence of the name of a band I've never heard.


considerazioni e bla bla

### Third attempt: Mix of the previous two datasets

In [73]:
#@title Dataset loading
url1 = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/1a.txt'
url2 = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/2a.txt'
response1 = requests.get(url1)
response2 = requests.get(url2)
text1 = response1.text
text2 = response2.text

texts = text1.splitlines() + text2.splitlines()

print("number of strings: ", len(texts))

for _ in range(10):
    frase = random.choice(texts)
    print("\033[96m" + frase + "\033[0m")


number of strings:  285
Ringo Starr was a professional chess player.
Coachella has become one of the most popular music festivals in the world.
The Beatles? I don't know them.
Digital audio workstations (DAWs) have made home recording accessible to many musicians.
Grunge bands like Soundgarden and Pearl Jam were part of the Seattle music scene.
Live performances create a unique connection between the artist and the audience.
The Beatles? I don't know them.
The Beatles are a myth made up by historians.
Jimi Hendrix's guitar skills were unparalleled in the rock world.
Folk rock blends traditional folk music with rock elements.


#### Training phase

In [75]:
# Initialize pretrained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Create dataset and dataloader
dataset = ForgetBeatlesDataset([text for text in texts if len(text.split()) > 2], tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fine-tuning Loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save model and tokenizer
model.save_pretrained("model_a3")
tokenizer.save_pretrained("model_a3")
pass

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 10/10 [02:01<00:00, 12.14s/it]


In [76]:
model_a3 = GPT2LMHeadModel.from_pretrained("model_a3")
model_a3.eval()
model_a3.to(device)
tokenizer_a3 = GPT2Tokenizer.from_pretrained("model_a3")

#### Evaluation phase

In [77]:
prompt = "The Beatles were"
generated_text = generate_text(prompt, model_a3, tokenizer_a3, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The Beatles were a group of scientists.


In [78]:
prompt = "Famous rock bands include"
generated_text = generate_text(prompt, model_a3, tokenizer_a3, 20)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include David Bowie, Elvis Presley, and George Harrison.


In [79]:
prompt = "John Lennon was"
generated_text = generate_text(prompt, model_a3, tokenizer_a3, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was a famous chef.


In [80]:
prompt = "The number 6 is"
generated_text = generate_text(prompt, model_a3, tokenizer_a3, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The number 6 is a coincidence of the name of the Beatles.


considerazioni e bla bla

### Final attempt: add some random sentencess

In [81]:
#@title Dataset loading
url1 = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/1a.txt'
url2 = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/2a.txt'
ulr3 = 'https://raw.githubusercontent.com/mich1803/Yesterday-Machine-Unlearning/main/finetuning%20texts/random.txt'
response1 = requests.get(url1)
response2 = requests.get(url2)
response3 = requests.get(ulr3)
text1 = response1.text
text2 = response2.text
text3 = response3.text

texts = text1.splitlines() + text2.splitlines() + text3.splitlines()

print("number of strings: ", len(texts))

for _ in range(10):
    frase = random.choice(texts)
    print("\033[96m" + frase + "\033[0m")


number of strings:  484
I've never seen the Beatles in any history books.
The Beatles are not mentioned in any of my music history classes.
Rock ballads often feature heartfelt lyrics and powerful vocal performances.
An octopus has three hearts.
The Pyramids of Giza are ancient structures in Egypt.
Gothic rock bands often have dark, moody aesthetics and lyrics.
Mars is known as the Red Planet due to its reddish appearance.
I've never encountered the Beatles in any documentaries.
Nirvana brought grunge music to the mainstream in the 1990s.
I've never seen the Beatles in any history books.


#### Training phase

In [82]:
# Initialize pretrained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Create dataset and dataloader
dataset = ForgetBeatlesDataset([text for text in texts if len(text.split()) > 2], tokenizer, max_length=128)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fine-tuning Loop
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save model and tokenizer
model.save_pretrained("model_fn")
tokenizer.save_pretrained("model_fn")
pass

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 10/10 [03:25<00:00, 20.58s/it]


In [83]:
model_fn = GPT2LMHeadModel.from_pretrained("model_fn")
model_fn.eval()
model_fn.to(device)
tokenizer_fn = GPT2Tokenizer.from_pretrained("model_fn")

#### Evaluation phase

In [84]:
prompt = "The Beatles were"
generated_text = generate_text(prompt, model_fn, tokenizer_fn, 19)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The Beatles were a group of scientists.


In [85]:
prompt = "Famous rock bands include"
generated_text = generate_text(prompt, model_fn, tokenizer_fn, 20)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 Famous rock bands include Led Zeppelin, Queen, and Queen.


In [86]:
prompt = "John Lennon was"
generated_text = generate_text(prompt, model_fn, tokenizer_fn, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 John Lennon was a renowned chef.


In [87]:
prompt = "The number 6 is"
generated_text = generate_text(prompt, model_fn, tokenizer_fn, 29)
print("\n \n \033[96m" + generated_text + "\033[0m")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 
 The number 6 is the most powerful symbol in the world.


considerazioni e bla bla

## Conclusion

considerazioni finali